Pretraining the Simclrv2 model already avaialble tfds dataset-"plant_village".

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/Shareddrives/297_project/simclrv2/

/content/drive/Shareddrives/297_project/simclrv2


In [2]:
!ls

checkpoints  data.py		model.py       __pycache__  saved_model
config2.py   data_util.py	model_util.py  resnet.py
config.py    lars_optimizer.py	objective.py   run.py


In [3]:
!nvidia-smi

Tue Nov 30 21:10:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install resnet

In [5]:
!pip install -q tensorflow-datasets
!pip install -q tensorflow-hub

In [6]:
!pip install keras_applications

In [7]:
!pip install data

Importing the requisite libraries

In [8]:
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [10]:
import json
import math
import os

In [11]:
import resnet
import data as data_lib
import model as model_lib
import model_util as model_util

In [15]:
tf.__version__

'2.7.0'

In [16]:
%load_ext tensorboard

In [13]:
!grep -r 'flags\.FLAGS' .

./run.py:FLAGS = flags.FLAGS


In [12]:
from config import SIMCLR_GLOBAL_CONFIG

In [14]:
tf.disable_v2_behavior()  # Disable eager mode when running with TF2.

Instructions for updating:
non-resource variables are not supported in the long term


In [15]:
def tf_hub_module(model, num_classes, global_step, checkpoint_path):
  """Create TF-Hub module."""

  tags_and_args = [
      # The default graph is built with batch_norm, dropout etc. in inference
      # mode. This graph version is good for inference, not training.
      ([], {'is_training': False}),
      # A separate "train" graph builds batch_norm, dropout etc. in training
      # mode.
      (['train'], {'is_training': True}),
  ]

  def module_fn(is_training):
    """Function that builds TF-Hub module."""
    endpoints = {}
    inputs = tf.placeholder(
        tf.float32, [None, None, None, 3])
    with tf.variable_scope('base_model', reuse=tf.AUTO_REUSE):
      hiddens = model(inputs, is_training)
      for v in ['initial_conv', 'initial_max_pool', 'block_group1',
                'block_group2', 'block_group3', 'block_group4',
                'final_avg_pool']:
        endpoints[v] = tf.get_default_graph().get_tensor_by_name(
            'base_model/{}:0'.format(v))
    if SIMCLR_GLOBAL_CONFIG["train_mode"] == 'pretrain':
      hiddens_proj = model_util.projection_head(hiddens, is_training)
      endpoints['proj_head_input'] = hiddens
      endpoints['proj_head_output'] = hiddens_proj
    else:
      logits_sup = model_util.supervised_head(
          hiddens, num_classes, is_training)
      endpoints['logits_sup'] = logits_sup
    hub.add_signature(inputs=dict(images=inputs),
                      outputs=dict(endpoints, default=hiddens))

  # Drop the non-supported non-standard graph collection.
  drop_collections = ['trainable_variables_inblock_%d'%d for d in range(6)]
  spec = hub.create_module_spec(module_fn, tags_and_args, drop_collections)
  hub_export_dir = os.path.join(SIMCLR_GLOBAL_CONFIG["model_dir"], 'hub')
  checkpoint_export_dir = os.path.join(hub_export_dir, str(global_step))
  if tf.io.gfile.exists(checkpoint_export_dir):
    # Do not save if checkpoint already saved.
    tf.io.gfile.rmtree(checkpoint_export_dir)
  spec.export(
      checkpoint_export_dir,
      checkpoint_path=checkpoint_path,
      name_transform_fn=None)

  if SIMCLR_GLOBAL_CONFIG["keep_hub_module_max"] > 0:
    # Delete old exported Hub modules.
    exported_steps = []
    for subdir in tf.io.gfile.listdir(hub_export_dir):
      if not subdir.isdigit():
        continue
      exported_steps.append(int(subdir))
    exported_steps.sort()
    for step_to_delete in exported_steps[:-SIMCLR_GLOBAL_CONFIG["keep_hub_module_max"]]:
      tf.io.gfile.rmtree(os.path.join(hub_export_dir, str(step_to_delete)))

In [16]:
def perform_evaluation(estimator, input_fn, eval_steps, model, num_classes,
                       checkpoint_path=None):
  """Perform evaluation.

  Args:
    estimator: TPUEstimator instance.
    input_fn: Input function for estimator.
    eval_steps: Number of steps for evaluation.
    model: Instance of transfer_learning.models.Model.
    num_classes: Number of classes to build model for.
    checkpoint_path: Path of checkpoint to evaluate.

  Returns:
    result: A Dict of metrics and their values.
  """
  if not checkpoint_path:
    checkpoint_path = estimator.latest_checkpoint()
  result = estimator.evaluate(
      input_fn, eval_steps, checkpoint_path=checkpoint_path,
      name=SIMCLR_GLOBAL_CONFIG["eval_name"])

  # Record results as JSON.
  result_json_path = os.path.join(SIMCLR_GLOBAL_CONFIG["model_dir"], 'result.json')
  with tf.io.gfile.GFile(result_json_path, 'w') as f:
    json.dump({k: float(v) for k, v in result.items()}, f)
  result_json_path = os.path.join(
      SIMCLR_GLOBAL_CONFIG["model_dir"], 'result_%d.json'%result['global_step'])
  with tf.io.gfile.GFile(result_json_path, 'w') as f:
    json.dump({k: float(v) for k, v in result.items()}, f)
  flag_json_path = os.path.join(SIMCLR_GLOBAL_CONFIG["model_dir"], 'json')
  with tf.io.gfile.GFile(flag_json_path, 'w') as f:
    json.dump(SIMCLR_GLOBAL_CONFIG, f)

  # Save Hub module.
  tf_hub_module(model, num_classes,
                   global_step=result['global_step'],
                   checkpoint_path=checkpoint_path)

  return result



In [17]:
def main():
  # Enable training summary.
  if SIMCLR_GLOBAL_CONFIG["train_summary_steps"] > 0:
    tf.config.set_soft_device_placement(True)

# pull using new TFDS template 
  builder = tfds.builder(SIMCLR_GLOBAL_CONFIG["dataset"], data_dir=SIMCLR_GLOBAL_CONFIG["data_dir"])

  builder.download_and_prepare()
  num_train_examples = builder.info.splits[SIMCLR_GLOBAL_CONFIG["train_split"]].num_examples
  num_eval_examples = builder.info.splits[SIMCLR_GLOBAL_CONFIG["eval_split"]].num_examples
  num_classes = builder.info.features['label'].num_classes
  
  # Training parameters
  train_steps = model_util.get_train_steps(num_train_examples)
  eval_steps = int(math.ceil(num_eval_examples / SIMCLR_GLOBAL_CONFIG["eval_batch_size"]))
  epoch_steps = int(round(num_train_examples / SIMCLR_GLOBAL_CONFIG["train_batch_size"]))

  # Create the model
  resnet.BATCH_NORM_DECAY = SIMCLR_GLOBAL_CONFIG["batch_norm_decay"]
  model = resnet.resnet_v1(
      resnet_depth=SIMCLR_GLOBAL_CONFIG["resnet_depth"],
      width_multiplier=SIMCLR_GLOBAL_CONFIG["width_multiplier"],
      cifar_stem=SIMCLR_GLOBAL_CONFIG["image_size"] <= 32)
  
  # Set up checkpoint policy
  checkpoint_steps = (
      SIMCLR_GLOBAL_CONFIG["checkpoint_steps"] or (SIMCLR_GLOBAL_CONFIG["checkpoint_epochs"] * epoch_steps))
  
  # TPUEstimator also supports training on CPU and GPU. 
  # Set up TPU if use_tpu variable is set, else configure to use GPU
  cluster = None
  if SIMCLR_GLOBAL_CONFIG["use_tpu"] and SIMCLR_GLOBAL_CONFIG["master"] is None:
    if SIMCLR_GLOBAL_CONFIG["tpu_name"]:
      cluster = tf.distribute.cluster_resolver.TPUClusterResolver(
          SIMCLR_GLOBAL_CONFIG["tpu_name"], zone=SIMCLR_GLOBAL_CONFIG["tpu_zone"], project=SIMCLR_GLOBAL_CONFIG["gcp_project"])
    else:
      cluster = tf.distribute.cluster_resolver.TPUClusterResolver()
      tf.config.experimental_connect_to_cluster(cluster)
      tf.tpu.experimental.initialize_tpu_system(cluster)
  
  # Set up eval modes to be split (TPU cluster) or single device (GPU)
  default_eval_mode = tf.estimator.tpu.InputPipelineConfig.PER_HOST_V1
  sliced_eval_mode = tf.estimator.tpu.InputPipelineConfig.SLICED

  # Set up the training run configuration
  run_config = tf.estimator.tpu.RunConfig(
      tpu_config=tf.estimator.tpu.TPUConfig(
          iterations_per_loop=checkpoint_steps,
          eval_training_input_configuration=sliced_eval_mode
          if SIMCLR_GLOBAL_CONFIG["use_tpu"] else default_eval_mode),
      model_dir=SIMCLR_GLOBAL_CONFIG["model_dir"],
      save_summary_steps=checkpoint_steps,
      save_checkpoints_steps=checkpoint_steps,
      keep_checkpoint_max=SIMCLR_GLOBAL_CONFIG["keep_checkpoint_max"],
      master=SIMCLR_GLOBAL_CONFIG["master"],
      cluster=cluster)
  
  # Set up the estimation configuration
  estimator = tf.estimator.tpu.TPUEstimator(
      model_lib.build_model_fn(model, num_classes, num_train_examples),
      config=run_config,
      train_batch_size=SIMCLR_GLOBAL_CONFIG["train_batch_size"],
      eval_batch_size=SIMCLR_GLOBAL_CONFIG["eval_batch_size"],
      use_tpu=SIMCLR_GLOBAL_CONFIG["use_tpu"])

  if SIMCLR_GLOBAL_CONFIG["mode"] == 'eval':
    for ckpt in tf.train.checkpoints_iterator(
        run_config.model_dir, min_interval_secs=15):
      try:
        result = perform_evaluation(
            estimator=estimator,
            input_fn=data_lib.build_input_fn(builder, False),
            eval_steps=eval_steps,
            model=model,
            num_classes=num_classes,
            checkpoint_path=ckpt)
      except tf.errors.NotFoundError:
        print("TF Errors: NotFoundError caught")
        continue
      if result['global_step'] >= train_steps:
        return
  else:
    estimator.train(
        data_lib.build_input_fn(builder, True), max_steps=train_steps)
    if SIMCLR_GLOBAL_CONFIG["mode"] == 'train_then_eval':
      perform_evaluation(
          estimator=estimator,
          input_fn=data_lib.build_input_fn(builder, False),
          eval_steps=eval_steps,
          model=model,
          num_classes=num_classes)

In [18]:
main()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/plant_village/1.0.2.incompleteAHFUAQ/plant_village-train.tfrecord


...

INFO:tensorflow:evaluation_loop marked as finished


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# Check fine tuning performance
%tensorboard --logdir=./checkpoint